In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/test.csv


In [2]:
# Import the libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim

In [3]:
# Load the data
train_data = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
valid_data = pd.read_csv("/kaggle/input/Kannada-MNIST/Dig-MNIST.csv")

In [4]:
class KannadaMNIST(Dataset):
    def __init__(self,data):
        self.data = data
        self.img_labels = self.data.iloc[:,0]
        self.img = self.data.iloc[:,1:]
        
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self,idx):
        image = self.img.iloc[idx,:].values.reshape(1,28,28)
        label = self.img_labels.iloc[idx]
        return image,label

In [5]:
train_dataset = KannadaMNIST(train_data)
valid_dataset = KannadaMNIST(valid_data)

In [6]:
batch_size=64
train_dataloader = DataLoader(train_dataset,batch_size = batch_size,shuffle = True)
valid_dataloader = DataLoader(valid_dataset,batch_size = batch_size,shuffle = True)
a,b = next(iter(train_dataloader))
a.data.shape

torch.Size([64, 1, 28, 28])

In [7]:
class MNISTConvNet(nn.Module):
    def __init__(self):
        super(MNISTConvNet, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
        self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, 5, padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
        self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(3136, 1024),
        nn.Dropout(0.5),
        nn.Linear(1024, 10)
    )
    def forward(self, x):
        x = x.float()
        x = self.conv1(x)
        x = self.conv2(x)
        return self.fc1(x)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MNISTConvNet().to(device)
model

MNISTConvNet(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=1024, bias=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [10]:
lr = 0.001
num_epochs = 20

loss_fn = nn.CrossEntropyLoss()
optimizer=optim.AdamW(model.parameters(),lr=lr)

In [12]:
for epochs in range(num_epochs):
    running_loss = 0.0
    num_correct = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
        _, idx = outputs.max(dim=1)
        num_correct += (idx == labels).sum().item()
print('Loss: {} Accuracy: {}'.format(running_loss/len(train_dataloader),
                                 num_correct/len(train_dataloader)))

Loss: 0.1030041851500137 Accuracy: 63.65138592750533
